In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/home/erik/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_data= datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
train_data1, train_data2 = torch.utils.data.random_split(training_data, [30000, 30000])


In [3]:
batch_size = 64
train_dataloader1 = DataLoader(train_data1, batch_size=batch_size)
train_dataloader2 = DataLoader(train_data2, batch_size=batch_size)
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X,y in train_dataloader1:
    print(f"Shape of X [N,C,W,H]: {X.shape}")
    print(f"Shape of y: {y.shape} ")
    break

Shape of X [N,C,W,H]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) 


In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self,x):
        x= self.flatten(x)
        logits = nn.functional.softmax(self.linear_relu_stack(x))
        return logits

model = NeuralNetwork().to(device)
print(model)



NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def real_train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
for (X,y),(X1,y1) in zip(train_dataloader1, train_dataloader2):
    print(X.shape)
    print(X1.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])


In [9]:
def train(dataloader1, dataloader2, model, loss_fn, optimizer):
    size1 = len(dataloader1.dataset)
    size2 = len(dataloader2.dataset)
    model.train()
    for batch, ((X,y),(X1,y1)) in enumerate(zip(dataloader1, dataloader2)):
        X, y = X.to(device), y.to(device)
        X1 = X1.to(device)
        # prediction 1
        pred1 = model(X)
        pred2 = model(X1)  # with label
        loss = loss_fn(pred1, y)
        # Backpropagation 1
        optimizer.zero_grad()
        loss.backward(retain_graph=True)

        # prediction 2
        pred3 = model(X1)  # without label
        loss2 = loss_fn(pred2, pred3)
        # Backpropagation 2
        loss2.backward()

        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print('for X')
            print(f"loss: {loss} [{current:>5d}/{size1:>5d}]")
        if batch % 100 == 0:
            loss2, current = loss2.item(), (batch+1)*len(X1)
            print('for X1')
            print(f"loss: {loss2} [{current:>5d}/{size2:>5d}]")


In [194]:
def train2(dataloader, model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        pred = model(X)
        print(pred)
        loss = loss_fn(pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break
        

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------")
    # train(train_dataloader1, train_dataloader2, model, loss_fn, optimizer)
    real_train(train_dataloader1, model,loss_fn,optimizer)
    test(test_dataloader, model, loss_fn)
print("done!")


Epoch 1
----------------------------------
loss: 2.302665  [   64/30000]


/tmp/ipykernel_28884/3805330871.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = nn.functional.softmax(self.linear_relu_stack(x))


loss: 2.301972  [ 6464/30000]
loss: 2.301989  [12864/30000]
loss: 2.302199  [19264/30000]
loss: 2.302210  [25664/30000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.302178 

Epoch 2
----------------------------------
loss: 2.302665  [   64/30000]
loss: 2.301972  [ 6464/30000]
loss: 2.301989  [12864/30000]
loss: 2.302199  [19264/30000]
loss: 2.302210  [25664/30000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.302178 

Epoch 3
----------------------------------
loss: 2.302665  [   64/30000]
loss: 2.301972  [ 6464/30000]
loss: 2.301989  [12864/30000]
loss: 2.302199  [19264/30000]
loss: 2.302210  [25664/30000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.302178 

Epoch 4
----------------------------------
loss: 2.302665  [   64/30000]
loss: 2.301972  [ 6464/30000]
loss: 2.301989  [12864/30000]
loss: 2.302199  [19264/30000]
loss: 2.302210  [25664/30000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 2.302178 

Epoch 5
----------------------------------
loss: 2.302665  [   64/30000]
loss: 2.301972  [ 64

Saving models

In [39]:
torch.save(model.state_dict(),"./models/model.pth")
print("Saved Pytorch Model State to model.pth")

Saved Pytorch Model State to model.pth


Loading models

In [41]:
model = NeuralNetwork()
model.load_state_dict(torch.load("./models/model.pth"))

<All keys matched successfully>

In [45]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]
model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
